# Text-to-SQL with Vanna.ai - Complete Tutorial

**Duration:** 1-2 hours  
**Level:** Intermediate (requires SQL knowledge and basic LLM understanding)  
**Framework:** Vanna.ai 2.0 Agent Framework  
**Database:** PostgreSQL (Supabase)

---

## What You'll Learn

1. What Text-to-SQL is and why it matters
2. How to set up Vanna.ai 2.0 with OpenAI and PostgreSQL
3. Understanding the Agent framework architecture
4. Generating SQL from natural language questions
5. Executing queries and handling results
6. Best practices for production deployment

---

## Prerequisites

- OpenAI API key
- PostgreSQL database (Supabase or local)
- Python 3.12+
- Basic understanding of SQL and LLMs

---

# Section 1: Introduction to Text-to-SQL

## What is Text-to-SQL?

Text-to-SQL converts natural language questions into SQL queries:

```
Question: "How many customers do we have?"
         ↓
SQL: SELECT COUNT(*) FROM customers
         ↓
Result: 100
```

## Why Use Text-to-SQL?

1. **Democratize Data Access** - Non-technical users can query databases
2. **Faster Analytics** - No manual SQL writing required
3. **Business Intelligence** - Power chatbots and dashboards
4. **Real-World Use Cases**: Slack bots, analytics dashboards, customer support tools

## Technical Architecture

```
Natural Language Question
         ↓
    LLM (OpenAI GPT-4o)
         ↓
    SQL Generation
         ↓
    PostgreSQL Database
         ↓
    Results (DataFrame)
```

## Vanna.ai 2.0 Overview

- **Agent Framework** - Modern architecture with user awareness
- **Modular Design** - Swap LLMs, databases, tools
- **Security** - User permissions and access control
- **Streaming UI** - Real-time component updates

---

# Section 2: Environment Setup

## Install Dependencies

In [1]:
# Install required packages
#!pip install vanna openai psycopg2-binary python-dotenv pandas sqlalchemy -q

## Import Libraries

**Important:** Vanna 2.0 uses different imports than Legacy (0.x)

In [2]:
# Core Vanna 2.0 imports
from vanna import Agent
from vanna.integrations.openai import OpenAILlmService
from vanna.integrations.postgres import PostgresRunner
from vanna.core.registry import ToolRegistry
from vanna.tools import RunSqlTool
from vanna.core.user import UserResolver, User, RequestContext
from vanna.integrations.local.agent_memory import DemoAgentMemory

# Standard libraries
import os
import pandas as pd
import psycopg2
from dotenv import load_dotenv
from urllib.parse import urlparse
from IPython.display import display

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Load Environment Variables

In [3]:
# Load from .env file
load_dotenv()

# Get credentials
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_URL = os.getenv("DATABASE_URL")

# Verify credentials exist
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment")
if not DATABASE_URL:
    raise ValueError("DATABASE_URL not found in environment")

print("✓ Environment variables loaded")
print(f"  OpenAI API Key: {OPENAI_API_KEY[:20]}...")
print(f"  Database: {DATABASE_URL.split('@')[1] if '@' in DATABASE_URL else 'configured'}")

✓ Environment variables loaded
  OpenAI API Key: sk-sjnjPsGndXBRy_3Bc...
  Database: db.jjegzpclybjhtzbjasze.supabase.co:5432/postgres


---

# Section 3: Initialize Vanna 2.0 Agent

## Architecture Components

Vanna 2.0 uses a modular Agent architecture:

1. **LlmService** - The language model (OpenAI GPT-4o)
2. **SqlRunner** - Database connection and execution
3. **ToolRegistry** - Available tools for the Agent
4. **UserResolver** - User authentication and permissions
5. **AgentMemory** - Conversation history
6. **Agent** - Orchestrates everything

## Step 1: Initialize LLM Service

In [4]:
# Initialize OpenAI GPT-4o
llm = OpenAILlmService(
    api_key=OPENAI_API_KEY,
    model="gpt-4o"  # Latest OpenAI model
)

print("✓ LLM Service initialized (OpenAI GPT-4o)")

✓ LLM Service initialized (OpenAI GPT-4o)


## Step 2: Initialize PostgreSQL Runner

**Important:** Use `connection_string` approach (simplest)

In [5]:
# Connect to PostgreSQL using connection string
postgres_runner = PostgresRunner(
    connection_string=DATABASE_URL
)

print("✓ PostgreSQL Runner initialized")

✓ PostgreSQL Runner initialized


## Step 3: Register Tools

Tools define what the Agent can do. We'll register the `RunSqlTool`.

In [6]:
# Create tool registry
tools = ToolRegistry()

# Register SQL execution tool
tools.register_local_tool(
    RunSqlTool(sql_runner=postgres_runner),
    access_groups=['user', 'admin']  # Who can use this tool
)

print("✓ Tools registered (RunSqlTool)")

✓ Tools registered (RunSqlTool)


## Step 4: Create User Resolver

User resolver handles authentication and permissions.

In [7]:
class SimpleUserResolver(UserResolver):
    """Simple user resolver for tutorial purposes"""
    
    async def resolve_user(self, request_context: RequestContext) -> User:
        return User(
            id="tutorial_user",
            email="student@tutorial.com",
            group_memberships=['user', 'admin']  # Full access
        )

user_resolver = SimpleUserResolver()

print("✓ User Resolver created")

✓ User Resolver created


## Step 5: Initialize Agent

The Agent orchestrates all components.

In [8]:
# Create the Agent
agent = Agent(
    llm_service=llm,
    tool_registry=tools,
    user_resolver=user_resolver,
    agent_memory=DemoAgentMemory()  # In-memory conversation history
)

print("✓ Agent initialized successfully!")
print("\n=== Vanna 2.0 Agent Ready ===")

✓ Agent initialized successfully!

=== Vanna 2.0 Agent Ready ===


---

# Section 4: Understanding the Database

## Database Schema Overview

Our e-commerce database has 3 tables:

### 1. `customers` (100 rows)
- `id` - Primary key
- `name` - Customer name
- `email` - Email address
- `segment` - SMB, Enterprise, or Individual
- `country` - Customer country

### 2. `products` (50 rows)
- `id` - Primary key
- `name` - Product name
- `category` - Product category
- `price` - Product price
- `stock_quantity` - Inventory count

### 3. `orders` (200 rows)
- `id` - Primary key
- `customer_id` - Foreign key to customers
- `order_date` - Order date
- `total_amount` - Order total (NOT 'price'!)
- `status` - Pending, Delivered, Cancelled, Processing

### Relationships
- `customers.id` → `orders.customer_id` (one-to-many)

## Create Helper Function for Direct SQL

Sometimes we want to run SQL directly without the Agent.

In [9]:
def run_sql_simple(sql: str) -> pd.DataFrame:
    """
    Execute SQL directly using psycopg2 (non-async, simple).
    
    Args:
        sql: SQL query string
        
    Returns:
        DataFrame with results
    """
    # Parse DATABASE_URL
    parsed = urlparse(DATABASE_URL)
    
    # Connect to database
    conn = psycopg2.connect(
        host=parsed.hostname,
        database=parsed.path[1:],  # Remove leading '/'
        user=parsed.username,
        password=parsed.password,
        port=parsed.port or 5432
    )
    
    # Execute query
    df = pd.read_sql_query(sql, conn)
    conn.close()
    
    return df

print("✓ Helper function created: run_sql_simple()")

✓ Helper function created: run_sql_simple()


## Preview Database Tables

In [10]:
# Preview customers table
print("=== CUSTOMERS TABLE (Sample) ===")
customers_sample = run_sql_simple("SELECT * FROM customers LIMIT 5")
display(customers_sample)

=== CUSTOMERS TABLE (Sample) ===


C:\Users\Dell\AppData\Local\Temp\ipykernel_14692\4022728488.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,id,name,email,segment,country,created_at,updated_at
0,1,Jason Bridges,rachel99@example.org,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
1,2,Andrew Warren,anne17@example.net,Enterprise,UK,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
2,3,Allison Howell,sextonashley@example.org,SMB,Germany,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
3,4,Sarah Miller,jonesbrittney@example.org,Enterprise,Canada,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
4,5,Lacey Gregory,angelamartin@example.com,Individual,Canada,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761


In [11]:
# Preview products table
print("=== PRODUCTS TABLE (Sample) ===")
products_sample = run_sql_simple("SELECT * FROM products LIMIT 5")
display(products_sample)

=== PRODUCTS TABLE (Sample) ===


C:\Users\Dell\AppData\Local\Temp\ipykernel_14692\4022728488.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,id,name,category,price,stock_quantity,description,created_at,updated_at
0,1,"Design Book Stewart, Strong and Diaz",Books,1903.20,411,Natural smile safe leg. Key guess car agreemen...,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
1,2,"Support Plan Jordan, Mcdonald and Davenport",Services,1457.39,11,Real perhaps foot receive technology. Attentio...,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
2,3,Adapter,Hardware,684.03,83,Really maybe however seat media. Movie less De...,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
3,4,CRM System,Software,175.33,460,Same college yeah general happen.\nGuy unit ne...,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
4,5,Router,Hardware,839.15,279,None,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761


In [12]:
# Preview orders table
print("=== ORDERS TABLE (Sample) ===")
orders_sample = run_sql_simple("SELECT * FROM orders LIMIT 5")
display(orders_sample)

=== ORDERS TABLE (Sample) ===


C:\Users\Dell\AppData\Local\Temp\ipykernel_14692\4022728488.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,id,customer_id,order_date,total_amount,status,shipping_address,created_at,updated_at
0,1,69,2026-01-04,3867.50,Delivered,"061 Becky Mews\nEast Jeffmouth, NM 19923",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
1,2,22,2025-12-11,2527.51,Delivered,None,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
2,3,50,2025-10-26,2442.76,Delivered,USCGC Little\nFPO AP 38776,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
3,4,87,2025-09-02,1979.80,Delivered,"1309 Jennifer Station\nWest Tara, SD 05107",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
4,5,41,2025-09-23,4187.62,Delivered,"3515 Evans Orchard\nLake Sierraport, WY 47029",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761


## Database Statistics

In [13]:
# Get table counts
stats = run_sql_simple("""
SELECT 
    (SELECT COUNT(*) FROM customers) as total_customers,
    (SELECT COUNT(*) FROM products) as total_products,
    (SELECT COUNT(*) FROM orders) as total_orders,
    (SELECT COUNT(DISTINCT segment) FROM customers) as customer_segments,
    (SELECT COUNT(DISTINCT category) FROM products) as product_categories,
    (SELECT COUNT(DISTINCT status) FROM orders) as order_statuses
""")

print("=== DATABASE STATISTICS ===")
display(stats)

=== DATABASE STATISTICS ===


C:\Users\Dell\AppData\Local\Temp\ipykernel_14692\4022728488.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,total_customers,total_products,total_orders,customer_segments,product_categories,order_statuses
0,100,50,200,3,6,4


---

# Section 5: Provide Schema Documentation to Agent

## Why Schema Documentation Matters

The Agent needs to understand:
- What tables exist
- What columns each table has
- Data types and relationships
- Business terminology

**Without schema knowledge, the Agent will generate incorrect SQL!**

Example: It might look for a `price` column in `orders` when the correct column is `total_amount`.

## Provide Schema Context

We'll give the Agent a detailed schema description.

In [14]:
# Schema documentation
SCHEMA_CONTEXT = """
DATABASE SCHEMA:

Table: customers
Columns:
  - id (SERIAL PRIMARY KEY)
  - name (VARCHAR) - Customer full name
  - email (VARCHAR) - Customer email address
  - segment (VARCHAR) - One of: 'SMB', 'Enterprise', 'Individual'
  - country (VARCHAR) - Customer country
  - created_at (TIMESTAMP)
  - updated_at (TIMESTAMP)

Table: products
Columns:
  - id (SERIAL PRIMARY KEY)
  - name (VARCHAR) - Product name
  - category (VARCHAR) - Product category (Electronics, Software, Hardware, etc.)
  - price (DECIMAL) - Product unit price
  - stock_quantity (INT) - Current inventory count
  - description (TEXT)
  - created_at (TIMESTAMP)
  - updated_at (TIMESTAMP)

Table: orders
Columns:
  - id (SERIAL PRIMARY KEY)
  - customer_id (INT) - Foreign key to customers.id
  - order_date (DATE) - Date of order
  - total_amount (DECIMAL) - TOTAL ORDER PRICE (use this for revenue, NOT 'price'!)
  - status (VARCHAR) - One of: 'Pending', 'Delivered', 'Cancelled', 'Processing'
  - shipping_address (TEXT)
  - created_at (TIMESTAMP)
  - updated_at (TIMESTAMP)

RELATIONSHIPS:
  - customers.id → orders.customer_id (one-to-many)
  
IMPORTANT NOTES:
  - For order revenue/pricing, use orders.total_amount (NOT 'price')
  - Customer segments: 'SMB', 'Enterprise', 'Individual' (case-sensitive)
  - Order statuses: 'Pending', 'Delivered', 'Cancelled', 'Processing' (case-sensitive)
  - To join customers and orders: JOIN orders ON customers.id = orders.customer_id
"""

print("✓ Schema context prepared")
print("\nThis will help the Agent understand the database structure.")

✓ Schema context prepared

This will help the Agent understand the database structure.


---

# Section 6: Querying with the Agent

## Create Helper Function for Agent Queries

**Important:** Vanna 2.0's `agent.send_message()` returns an **async generator**, not a simple awaitable.

We must use `async for` to iterate through UI components.

In [15]:
# Create a request context (simulates HTTP request)
request_context = RequestContext()

# Test query
print("🧪 Testing Agent with: 'How many customers are in the database?'\n")

try:
    # Iterate over streaming results
    async for ui_component in agent.send_message(
        request_context=request_context,
        message="How many customers are in the database?"
    ):
        # Each ui_component is a piece of the response
          print(ui_component)

    print("\n✅ Agent responded successfully!")

except Exception as e:
      print(f"❌ Error: {e}")
      print("\nTroubleshooting:")
      print("  • Make sure DATABASE_URL is set correctly")
      print("  • Ensure the database has a 'customers' table")
      print("  • Check OPENAI_API_KEY is valid")

🧪 Testing Agent with: 'How many customers are in the database?'

timestamp='2026-01-29T01:11:55.663741' rich_component=StatusBarUpdateComponent(id='vanna-status-bar', type=<ComponentType.STATUS_BAR_UPDATE: 'status_bar_update'>, lifecycle=<ComponentLifecycle.CREATE: 'create'>, data={}, children=[], timestamp='2026-01-29T01:11:55.663701', visible=True, interactive=False, status='working', message='Processing your request...', detail='Analyzing query') simple_component=None
timestamp='2026-01-29T01:11:55.670456' rich_component=TaskTrackerUpdateComponent(id='vanna-task-tracker', type=<ComponentType.TASK_TRACKER_UPDATE: 'task_tracker_update'>, lifecycle=<ComponentLifecycle.CREATE: 'create'>, data={}, children=[], timestamp='2026-01-29T01:11:55.667780', visible=True, interactive=False, operation=<TaskOperation.ADD_TASK: 'add_task'>, task=Task(id='ae76f0a0-8c17-466b-97c4-939f777ec0e6', title='Load conversation context', description='Reading message history and user context', status='pending',

In [16]:
async def ask_agent(question: str, include_schema: bool = True):
    """
    Ask the Agent a question and display results.
    
    Args:
        question: Natural language question
        include_schema: Whether to include schema context (recommended: True)
    """
    # Prepend schema context to question
    if include_schema:
        full_message = f"{SCHEMA_CONTEXT}\n\nQUESTION: {question}"
    else:
        full_message = question
    
    # Create request context
    request_context = RequestContext()
    
    print(f"🤔 Question: {question}\n")
    
    # Send message to agent (returns async generator)
    async for component in agent.send_message(
        request_context=request_context,
        message=full_message
    ):
        # Extract the UI component
        rich_comp = component.rich_component
        
        # Handle different component types
        if hasattr(rich_comp, 'rows') and rich_comp.rows:
            # DataFrameComponent - extract data from 'rows' attribute
            df = pd.DataFrame(rich_comp.rows)
            print("📊 Results:")
            display(df)
            print()
        
        elif hasattr(rich_comp, 'text') and rich_comp.text:
            # RichTextComponent - display text
            print(f"💬 {rich_comp.text}\n")
        
        elif hasattr(rich_comp, 'sql') and rich_comp.sql:
            # SQL code component
            print(f"🔍 Generated SQL:\n{rich_comp.sql}\n")

print("✓ Helper function created: ask_agent()")

✓ Helper function created: ask_agent()


### With Explanations

In [17]:
# async def ask_agent(question: str):
#       """Ask agent and display results properly."""
#       print(f"❓ Question: {question}\n")

#       request_context = RequestContext()

#       print("="*80 + "\n")

#       async for component in agent.send_message(
#           request_context=request_context,
#           message=question
#       ):
#           rich_comp = component.rich_component

#           # Check for DataFrame component
#           if hasattr(rich_comp, 'type') and 'DATAFRAME' in str(rich_comp.type):
#               # ✅ Get data from 'rows' attribute!
#               if hasattr(rich_comp, 'rows') and rich_comp.rows:
#                   df = pd.DataFrame(rich_comp.rows)

#                   print("📊 Query Results:\n")
#                   from IPython.display import display
#                   display(df)
#                   print()

#           # Check for text explanation
#           elif hasattr(rich_comp, 'type') and 'TEXT' in str(rich_comp.type):
#               if hasattr(rich_comp, 'content') and rich_comp.content:
#                   print("💬 Explanation:\n")
#                   print(rich_comp.content)
#                   print()

#       print("="*80 + "\n")

## Example 1: Simple COUNT Query

In [18]:
await ask_agent("How many customers do we have?")

🤔 Question: How many customers do we have?

📊 Results:


,customer_count
0,100


## Example 2: Aggregation Query

In [19]:
await ask_agent("What is the total revenue from all orders?")

🤔 Question: What is the total revenue from all orders?

📊 Results:


,total_revenue
0,506635.08


## Example 3: Filtering Query

In [20]:
await ask_agent("How many orders have been delivered?")

🤔 Question: How many orders have been delivered?

📊 Results:


,delivered_orders_count
0,132


## Example 4: JOIN Query

In [21]:
await ask_agent("Show me the top 5 customers by total spending")

🤔 Question: Show me the top 5 customers by total spending

📊 Results:


,name,total_spending
0,Mr. Darin Howard MD,17305.06
1,Tiffany Moore,15075.58
2,Joan Anderson,14285.57
3,Kenneth Cantrell,14147.37
4,Jose Castillo,13685.04


## Example 5: GROUP BY with JOIN

In [22]:
await ask_agent("What is the average order value for each customer segment?")

🤔 Question: What is the average order value for each customer segment?

📊 Results:


,segment,average_order_value
0,Individual,2455.5222826086956522
1,Enterprise,2740.6796610169491525
2,SMB,2429.1210204081632653


## Example 6: Time-Based Filtering

In [23]:
await ask_agent("Show orders from the last 30 days")

🤔 Question: Show orders from the last 30 days

📊 Results:


,id,customer_id,order_date,total_amount,status,shipping_address,created_at,updated_at
0,1,69,2026-01-04,3867.50,Delivered,"061 Becky Mews\nEast Jeffmouth, NM 19923",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
1,26,45,2026-01-05,3925.34,Delivered,"6742 Jessica Summit\nNavarrochester, RI 98248",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
2,85,55,2026-01-14,3476.79,Delivered,"93980 Joshua Shoal\nMichaelberg, MO 44360",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
3,108,69,2026-01-14,4013.20,Delivered,"71918 Miller Brook\nEast Larryville, PR 19096",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
4,123,68,2026-01-12,4666.14,Delivered,"289 Elliott Estate\nLake Leonard, RI 03770",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
5,169,14,2026-01-02,2107.69,Delivered,"1382 Garcia River\nFisherstad, SC 20506",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
6,184,77,2026-01-26,1519.81,Processing,"30698 Scott Flat\nNew Brian, HI 32968",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761


## Example 7: Complex Business Question

In [24]:
await ask_agent("What is the total revenue from delivered orders placed by Enterprise customers?")

🤔 Question: What is the total revenue from delivered orders placed by Enterprise customers?

📊 Results:


,total_revenue
0,117828.50


---

# Section 7: Understanding How the Agent Works

## The Agent's Workflow

When you ask a question, here's what happens:

1. **Question Processing** - Agent receives your natural language question
2. **Schema Retrieval** - Agent uses the schema context we provided
3. **SQL Generation** - GPT-4o generates SQL based on the question and schema
4. **Tool Execution** - Agent calls `RunSqlTool` to execute the SQL
5. **Result Formatting** - Results are returned as UI components (DataFrameComponent)
6. **Response** - We extract and display the results

## Why Schema Context is Critical

Compare these two approaches:

### Without Schema Context:

In [25]:
# This might generate incorrect SQL (looking for 'price' instead of 'total_amount')
await ask_agent(
    "What is the total revenue from delivered orders?",
    include_schema=False  # No schema context
)

🤔 Question: What is the total revenue from delivered orders?

📊 Results:


,total_revenue
0,None


### With Schema Context:

In [26]:
# This will generate correct SQL (using 'total_amount')
await ask_agent(
    "What is the total revenue from delivered orders?",
    include_schema=True  # Schema context included
)

🤔 Question: What is the total revenue from delivered orders?

📊 Results:


,total_revenue
0,339168.50


## Verifying Agent Results

Let's verify the Agent's answer with direct SQL:

In [27]:
# Direct SQL query for verification
verification_result = run_sql_simple("""
SELECT 
    COUNT(*) as delivered_count,
    SUM(total_amount) as total_revenue
FROM orders
WHERE status = 'Delivered'
""")

print("=== VERIFICATION (Direct SQL) ===")
display(verification_result)

=== VERIFICATION (Direct SQL) ===


C:\Users\Dell\AppData\Local\Temp\ipykernel_14692\4022728488.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,delivered_count,total_revenue
0,132,339168.5


---

# Section 8: Testing Various Question Types

## Simple Queries

In [28]:
await ask_agent("How many products are in stock?")

🤔 Question: How many products are in stock?

📊 Results:


,in_stock_count
0,50


## Aggregation Queries

In [29]:
await ask_agent("What is the average product price?")

🤔 Question: What is the average product price?

📊 Results:


,average_product_price
0,840.7284000000000000


In [30]:
await ask_agent("What is the highest order value?")

🤔 Question: What is the highest order value?

📊 Results:


,highest_order_value
0,4990.00


## Filtering and Sorting

In [31]:
await ask_agent("Show me the 10 most expensive products")

🤔 Question: Show me the 10 most expensive products

📊 Results:


,id,name,price
0,21,Cable Joseph LLC,1907.81
1,43,Router,1904.34
2,1,"Design Book Stewart, Strong and Diaz",1903.20
3,50,Cable,1677.32
4,36,Adapter Williams Group,1607.13
5,11,Programming Guide,1590.24
6,32,Speaker,1539.23
7,40,Monitor Johnson Group,1481.89
8,2,"Support Plan Jordan, Mcdonald and Davenport",1457.39
9,39,Laptop Baxter-Smith,1441.26


In [32]:
await ask_agent("Which customers are from the USA?")

🤔 Question: Which customers are from the USA?

📊 Results:


,id,name,email,segment,country,created_at,updated_at
0,1,Jason Bridges,rachel99@example.org,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
1,18,Tim Adams,nicole86@example.com,Enterprise,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
2,23,Wendy James,adamfarmer@example.com,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
3,26,James Bush,chayden@example.com,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
4,45,Mrs. Mary Campbell DVM,edwin29@example.com,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
5,51,David James,ashleywilliams@example.org,SMB,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
6,54,Mariah Salinas,mark63@example.com,Enterprise,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
7,57,Ashley Clark,collinsdaniel@example.net,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
8,61,Michael Payne,xmendoza@example.com,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
9,64,Lisa Ruiz,schultztyler@example.com,Enterprise,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761


## Complex JOIN Queries

In [33]:
await ask_agent("How many orders has each customer segment placed?")

🤔 Question: How many orders has each customer segment placed?

📊 Results:


,segment,order_count
0,Individual,92
1,Enterprise,59
2,SMB,49


In [34]:
await ask_agent("Which customers have never placed an order?")

🤔 Question: Which customers have never placed an order?

📊 Results:


,id,name
0,2,Andrew Warren
1,6,Donna Bond
2,8,Robert Dunlap MD
3,9,Emily Poole
4,26,James Bush
5,44,Michael Barrett
6,51,David James
7,52,Theresa Sutton
8,62,Mrs. Stephanie Owens
9,63,Cindy Mercer


---

# Section 9: Error Handling and Best Practices

## Common Issues and Solutions

### Issue 1: Ambiguous Questions

**Bad:** "Show me the data" (What data? Which table?)

**Good:** "Show me all customers from the USA"

In [35]:
# This will likely generate better SQL
await ask_agent("Show me all customers from the USA")

🤔 Question: Show me all customers from the USA

📊 Results:


,id,name,email,segment,country,created_at,updated_at
0,1,Jason Bridges,rachel99@example.org,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
1,18,Tim Adams,nicole86@example.com,Enterprise,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
2,23,Wendy James,adamfarmer@example.com,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
3,26,James Bush,chayden@example.com,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
4,45,Mrs. Mary Campbell DVM,edwin29@example.com,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
5,51,David James,ashleywilliams@example.org,SMB,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
6,54,Mariah Salinas,mark63@example.com,Enterprise,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
7,57,Ashley Clark,collinsdaniel@example.net,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
8,61,Michael Payne,xmendoza@example.com,Individual,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
9,64,Lisa Ruiz,schultztyler@example.com,Enterprise,USA,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761


### Issue 2: Incorrect Column Names

**Problem:** Agent looks for 'price' in orders table (doesn't exist)

**Solution:** Provide clear schema documentation (we do this in `SCHEMA_CONTEXT`)

In [36]:
# Schema context prevents this error
await ask_agent("What is the total revenue?")  # Will correctly use 'total_amount'

🤔 Question: What is the total revenue?

📊 Results:


,total_revenue
0,506635.08


### Issue 3: Case Sensitivity

**Problem:** PostgreSQL string comparisons are case-sensitive

**Solution:** Document exact values in schema context

In [37]:
# This works because we documented 'Delivered' (capitalized) in schema
await ask_agent("Show delivered orders")

🤔 Question: Show delivered orders

📊 Results:


,id,customer_id,order_date,total_amount,status,shipping_address,created_at,updated_at
0,1,69,2026-01-04,3867.50,Delivered,"061 Becky Mews\nEast Jeffmouth, NM 19923",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
1,2,22,2025-12-11,2527.51,Delivered,None,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
2,3,50,2025-10-26,2442.76,Delivered,USCGC Little\nFPO AP 38776,2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
3,4,87,2025-09-02,1979.80,Delivered,"1309 Jennifer Station\nWest Tara, SD 05107",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
4,5,41,2025-09-23,4187.62,Delivered,"3515 Evans Orchard\nLake Sierraport, WY 47029",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
...,...,...,...,...,...,...,...,...
127,193,49,2025-12-29,3812.29,Delivered,"55897 Cesar Square\nTuckerchester, WY 34340",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
128,195,7,2025-11-02,1272.55,Delivered,"07254 Rose Falls\nLake Natasha, OR 09820",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
129,196,14,2025-08-25,2287.12,Delivered,"177 Julia Mountain\nVincentmouth, NM 46645",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761
130,198,34,2025-05-18,1895.88,Delivered,"2484 Gina Corner Apt. 030\nGardnermouth, WV 05638",2026-01-29 00:57:13.212761,2026-01-29 00:57:13.212761


## Best Practices

1. **Always provide schema context** - Critical for accuracy
2. **Be specific in questions** - Avoid ambiguous language
3. **Document exact values** - Include enum values, case sensitivity
4. **Verify results** - Compare Agent output with direct SQL
5. **Use descriptive column names** - Helps the LLM understand intent
6. **Include business logic** - Document relationships and constraints
7. **Test edge cases** - Empty results, NULLs, etc.
8. **Monitor and log** - Track query accuracy in production

## Production Considerations

1. **User Permissions** - Implement proper `UserResolver` with auth
2. **SQL Validation** - Block dangerous operations (DROP, DELETE without WHERE)
3. **Rate Limiting** - Prevent abuse
4. **Query Approval** - Show SQL to users before execution
5. **Monitoring** - Track failed queries for retraining
6. **Caching** - Cache common questions
7. **Row-Level Security** - Filter data based on user permissions
8. **API Key Management** - Never hardcode credentials

---

# Section 10: Next Steps and Resources

## What We've Learned

✅ Text-to-SQL fundamentals
✅ Vanna.ai 2.0 Agent architecture
✅ Setting up LLM, database, tools, and user resolver
✅ Importance of schema documentation
✅ Querying with natural language
✅ Handling async operations correctly
✅ Extracting results from UI components
✅ Best practices and error handling

## Key Takeaways

1. **Schema context is critical** - Without it, the Agent will generate incorrect SQL
2. **Vanna 2.0 uses async generators** - Use `async for` to iterate components
3. **UI components hold data in different attributes** - DataFrameComponent uses `rows`
4. **Direct SQL is useful** - For verification and simple queries
5. **Production requires more** - Permissions, validation, monitoring

## Advanced Topics to Explore

- **Custom Tools** - Create your own tools beyond RunSqlTool
- **Multi-Turn Conversations** - Agent memory and context
- **Different LLMs** - Try Claude, Llama, or other models
- **Different Databases** - MySQL, SQLite, Snowflake
- **Web Interface** - Vanna's `<vanna-chat>` component
- **Security** - Row-level security, query validation
- **Performance** - Caching, query optimization

## Resources

- **Vanna.ai Documentation**: https://docs.vanna.ai/
- **GitHub Repository**: https://github.com/vanna-ai/vanna
- **Examples**: https://github.com/vanna-ai/vanna/tree/main/examples
- **Community**: Discord, GitHub Discussions
- **API Reference**: https://docs.vanna.ai/api/

## Your Next Project Ideas

1. **Build a Slack Bot** - Answer data questions in Slack
2. **Create a Dashboard** - Visualize query results
3. **HR Analytics** - Query employee database
4. **Sales Intelligence** - Real-time sales queries
5. **Customer Support** - Automated data lookups

---

## Thank You!

You've completed the Vanna.ai Text-to-SQL tutorial. You now have the foundation to build your own Text-to-SQL applications!

**Happy coding! 🚀**